<a href="https://colab.research.google.com/github/g23ai2068/Capstone-Project/blob/g23ai2011_Naresh-G/Capstoen_Phase1_Problemstatement2_UsingGANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem statement 2: Data Augmentation and Synthesis

#Importing all dependencies and loading dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Loading Review Dataset
review_data=pd.read_csv("/content/drive/MyDrive/Trimester_3/Capstone_project/Datasets/reviews.csv")
review_data

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,V Jillian,Holly's place was great. It was exactly what I...
...,...,...,...,...,...,...
392954,1219622445222452442,1230654319092135704,2024-08-24,51979490,Solène,D
392955,1219622445222452442,1232036398314385400,2024-08-26,18919150,Kristopher,Great stay. Key vob stopped working but host a...
392956,1219785829994931401,1237167935539710683,2024-09-02,4338272,Anastasia,Great choice for staying! The apartment is ama...
392957,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful


In [3]:
review_data.shape

(392959, 6)

In [4]:
review_data.value_counts()

listing_id           id                   date        reviewer_id  reviewer_name  comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
958                  5977                 2009-07-23  15695        Edmund C       Our experience was, without a doubt, a five star experience. Holly and her husband, David, were the consummate hosts; friendly and accomodating while still honoring our privacy. The apartment was a charming layout with a full view and access to the home's garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, the Castro and Golden Gate Park. I can't wait for our next visit.\r<br/>\r<br/>Ted and Karen Wingerd                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1
30069424             624869237            2020-05-15  81954409     Janiqua        I’ll start with the good. This is a really great place as far as getting an apt in a nice area, with a garage parking spot that you can actually fit a car in! It is also full of all the amenities you could need. The bathrooms are GORGEOUS! You also share the W/D with the neighbor, but still very convenient. Edwin is very responsive and always answered in a timely manner. On the flip side, there were more spiders than I cared to see. The walls were pretty scuffed when we arrived as well in nearly every room, so we had to magic erase those ourselves unfortunately. But the worst part of this stay was the upstairs neighbor. Beware! He sings nearly everyday, at the top of his lungs, with complete disregard for the people below him. It wouldn’t be so bad if you couldn’t hear him so clearly, but unfortunately you can and that’s even with ear plugs in. In addition, it’s just a lot of other miscellaneous noise that became the norm and got old quick.  Keep in mind I stayed here for 3 mos, so if you’re in and out you might not experience all of these issues. Just wanted to give a heads up I wish I had.    1
30091565             353128547            2018-11-26  12429185     Daylon         Good spot for a quick weekend stay in a central part of the city!                                                                                                                                                                                                                                                                              

In [5]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392959 entries, 0 to 392958
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     392959 non-null  int64 
 1   id             392959 non-null  int64 
 2   date           392959 non-null  object
 3   reviewer_id    392959 non-null  int64 
 4   reviewer_name  392956 non-null  object
 5   comments       392889 non-null  object
dtypes: int64(3), object(3)
memory usage: 18.0+ MB


In [6]:
review_data.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [7]:
review_data.describe()

,listing_id,id,reviewer_id
count,3.929590e+05,3.929590e+05,3.929590e+05
mean,8.225927e+16,4.223165e+17,1.251442e+08
std,2.459846e+17,4.557533e+17,1.407015e+08
min,9.580000e+02,1.981000e+03,1.000000e+00
25%,4.359078e+06,2.791330e+08,1.992458e+07
50%,1.742188e+07,7.701426e+08,6.663371e+07
75%,3.680482e+07,8.646046e+17,1.818866e+08
max,1.234137e+18,1.237913e+18,5.984690e+08


#Importing NLTK for sentiment score analysis.

In [8]:
# Installing nltk
!pip install nltk

In [9]:
# Importing all the dependencies for nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [10]:
# Using vader, we are importing sentiment analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Load the sentiment analyzer
sent = SentimentIntensityAnalyzer()
review_final = pd.read_csv('/content/drive/MyDrive/Trimester_3/Capstone_project/Datasets/reviews.csv', usecols=['comments'])
review_final['comments'] = review_final['comments'].astype(str)

# Calculate polarity scores
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in review_final['comments']]
review_final['sentiment_score'] = polarity
review_final.head()

,comments,sentiment_score
0,"Our experience was, without a doubt, a five st...",0.96
1,Returning to San Francisco is a rejuvenating t...,0.98
2,We were very pleased with the accommodations a...,0.76
3,We highly recommend this accomodation and agre...,0.98
4,Holly's place was great. It was exactly what I...,0.96


In [11]:
#def preprocess_reviews(review_final):
 # reviews = review_final
  #reviews = review_final.str.replace(r'[^a-z\s]', '', regex=True)
  #return reviews

In [12]:
# review_final['comments'] = preprocess_reviews(review_final['comments'])
# review_final.head(100)

In [13]:
review_final['comments']

,comments
0,"Our experience was, without a doubt, a five st..."
1,Returning to San Francisco is a rejuvenating t...
2,We were very pleased with the accommodations a...
3,We highly recommend this accomodation and agre...
4,Holly's place was great. It was exactly what I...
...,...
392954,D
392955,Great stay. Key vob stopped working but host a...
392956,Great choice for staying! The apartment is ama...
392957,Wonderful


# Tokenizer

In [14]:
# Tokenizing the words in the comment section
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review_final['comments'])
total_words = len(tokenizer.word_index) + 1

print('Total words: ', total_words)
print(tokenizer.word_index)

Total words:  118059
{'and': 1, 'the': 2, 'a': 3, 'to': 4, 'was': 5, 'in': 6, 'is': 7, 'great': 8, 'we': 9, 'very': 10, 'for': 11, 'of': 12, 'place': 13, 'i': 14, 'stay': 15, 'with': 16, 'it': 17, 'br': 18, 'location': 19, 'you': 20, 'clean': 21, 'this': 22, 'our': 23, 'had': 24, 'were': 25, 'at': 26, 'would': 27, 'nice': 28, 'as': 29, 'host': 30, 'my': 31, 'comfortable': 32, 'but': 33, 'so': 34, 'room': 35, 'again': 36, 'that': 37, 'there': 38, 'on': 39, 'from': 40, 'recommend': 41, 'are': 42, 'easy': 43, 'everything': 44, 'us': 45, 'all': 46, 'home': 47, 'san': 48, 'perfect': 49, 'apartment': 50, 'well': 51, 'an': 52, 'here': 53, 'definitely': 54, 'really': 55, 'francisco': 56, 'space': 57, 'have': 58, 'if': 59, 'be': 60, 'neighborhood': 61, 'quiet': 62, 'good': 63, 'sf': 64, 'city': 65, 'super': 66, 'not': 67, 'close': 68, 'time': 69, 'house': 70, 'beautiful': 71, 'out': 72, 'also': 73, 'restaurants': 74, 'area': 75, 'just': 76, 'parking': 77, 'wonderful': 78, 'amazing': 79, '\r': 8

In [16]:
sequences = tokenizer.texts_to_sequences(review_final['comments'])

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = max([len(seq) for seq in sequences])
# You can set a fixed length here
X = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_numerical = scaler.fit_transform(review_final[[ 'sentiment_score']])

In [19]:
X_combined = np.concatenate([X, X_numerical], axis=1)
X_combined

array([[2.300e+01, 1.260e+02, 5.000e+00, ..., 0.000e+00, 0.000e+00,
        9.800e-01],
       [1.148e+03, 4.000e+00, 4.800e+01, ..., 0.000e+00, 0.000e+00,
        9.900e-01],
       [9.000e+00, 2.500e+01, 1.000e+01, ..., 0.000e+00, 0.000e+00,
        8.800e-01],
       ...,
       [8.000e+00, 8.370e+02, 1.100e+01, ..., 0.000e+00, 0.000e+00,
        9.750e-01],
       [7.800e+01, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        7.850e-01],
       [2.200e+01, 1.300e+01, 7.000e+00, ..., 0.000e+00, 0.000e+00,
        9.900e-01]])

In [20]:
real_labels = np.ones((X_combined.shape[0], 1))
fake_labels = np.zeros((X_combined.shape[0], 1))

In [21]:
print(f"Text Data Shape: {X.shape}, Numerical Data Shape: {X_numerical.shape}")
latent_dim = 100

Text Data Shape: (392959, 1008), Numerical Data Shape: (392959, 1)


# Generator and descriminator

In [22]:
#def build_generator(latent_dim, max_sequence_length, total_words, num_numerical_features):
   # model = Sequential()
   # model.add(Dense(256, input_dim=latent_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
   # model.add(LeakyReLU(alpha=0.2))
   # model.add(BatchNormalization(momentum=0.8))
   # model.add(Dense(512))
   # model.add(LeakyReLU(alpha=0.2))
   # model.add(BatchNormalization(momentum=0.8))
   # model.add(Dense(max_sequence_length * total_words, activation='softmax'))
   # model.add(Dense(256, activation='relu'))
   # model.add(Dense(num_numerical_features, activation='sigmoid'))
   # return model

In [35]:
#def build_generator(latent_dim, max_sequence_length, total_words, num_numerical_features):
  #  model = Sequential()
  #  model.add(Dense(256, input_dim=latent_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
   # model.add(LeakyReLU(alpha=0.2))
   # model.add(BatchNormalization(momentum=0.8))
   # model.add(Dense(256))
   # model.add(LeakyReLU(alpha=0.2))
   # model.add(BatchNormalization(momentum=0.8))
   # model.add(Dense(max_sequence_length * total_words))  # Remove softmax activation
  #  model.add(Dense(256, activation='relu'))
  #  model.add(Dense(num_numerical_features, activation='sigmoid'))
   # return model

In [39]:
def build_generator(latent_dim, max_sequence_length, total_words, num_numerical_features):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256)) # Reduced the output dimension to 256
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    #Instead of multiplying max_sequence_length and total_words directly,
    # which was causing memory issues, this change uses a fixed output size.
    #  Adjust the value (256 in this case) as needed to balance model performance and memory usage.
    model.add(Dense(256))  # Remove softmax activation and replaced with smaller output size
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_numerical_features, activation='sigmoid'))
    return model

In [40]:
def build_discriminator(max_sequence_length, total_words, num_numerical_features):
    model = Sequential()
    model.add(LSTM(512, input_shape=(max_sequence_length, total_words), return_sequences=True))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_numerical_features, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [41]:
def build_gan(generator, discriminator):
    discriminator.trainable = False  # We only train the generator here
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [42]:
from tensorflow.keras import initializers #Import initializers from tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, LSTM

generator = build_generator(latent_dim, max_sequence_length, total_words, X_numerical.shape[1])
discriminator = build_discriminator(max_sequence_length, total_words, X_numerical.shape[1])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
from tensorflow.keras.optimizers import Adam
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

In [47]:
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [48]:
def train_gan(generator, discriminator, gan, X_train, epochs=10000, batch_size=64, sample_interval=1000):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_reviews = X_train[idx]

        # Generate fake reviews (using the generator)
        noise = np.random.randn(half_batch, latent_dim)
        fake_reviews = generator.predict(noise)